In [70]:
import pandas as pd

In [19]:
# Read csv files
transactions = pd.read_csv("data/transactions_train.csv",parse_dates = ['t_dat'])
articles = pd.read_csv("data/articles.csv")
customers = pd.read_csv("data/customers.csv")

In [5]:
# Number of articles, number of customers and total volume change over the time

# aggregate the transaction by date
transaction_aggr = transactions.groupby(['t_dat']).nunique().reset_index()[['t_dat','customer_id','article_id']]

# create a column showing sum per user per day
transaction_aggr['sum'] = transactions.groupby(['t_dat']).sum().reset_index()[['price']]
transaction_aggr = transaction_aggr.rename(columns={"customer_id": "nr_customer",
                                                   "article_id":"nr_article",
                                                   "sum":"total_volume"})

# Save csv
transaction_aggr.to_csv("data/transaction_aggr.csv",index=False)

In [6]:
# Colors

# join transaction dataset with articles
transaction_article_colour = pd.merge(transactions[['article_id', 't_dat']], articles[['article_id', 'colour_group_name']], how = 'inner', on = ['article_id']).reset_index()

# colours articles by month
articles_transaction_color_aggr = transaction_article_colour.groupby(['t_dat','colour_group_name']).count().reset_index()[['t_dat','colour_group_name','article_id']]
articles_transaction_color_aggr['t_dat'] = pd.to_datetime(articles_transaction_color_aggr['t_dat'])

# Convert date into month and year
articles_transaction_color_aggr['month_year']=articles_transaction_color_aggr['t_dat'].dt.strftime('%m/%Y')
articles_transaction_color_aggr['year']= pd.DatetimeIndex(articles_transaction_color_aggr['t_dat']).year

# Save csv
articles_transaction_color_aggr.to_csv("data/articles_transaction_color_aggr.csv",index=False)

By season

In [8]:
# Get the month of each date
transactions['month'] =  pd.DatetimeIndex(transactions['t_dat']).month

# Define the seasons along the year
transactions.loc[(transactions["month"] >= 3) & (transactions["month"] <= 5) , "season"] = "Spring"
transactions.loc[(transactions["month"] >= 6) & (transactions["month"] <= 8) , "season"] = "Summer"
transactions.loc[(transactions["month"] >= 9) & (transactions["month"] <= 11) , "season"] = "Autumn"
transactions.loc[(transactions["month"] == 12) , "season"] = "Winter"
transactions.loc[(transactions["month"] >= 1) & (transactions["month"] <= 2) , "season"] = "Winter"

In [ ]:
# Products per season
transactions_season = transactions[['season' , 'article_id']].merge(articles[['article_id' , 'product_type_name' , 'product_type_no']] , on = 'article_id').groupby(['season' , 'product_type_no' ,'product_type_name']).agg({'product_type_no': 'count'}).rename(columns={'product_type_no': 'quantity'}).reset_index()

# Save csv
transactions_season.to_csv("data/transactions_season.csv",index=False)

In [13]:
# Age Group
import numpy as np
bins = np.array([0,15,19,35,50,99])
labels = ['unknown' , 'teens' , 'young' , 'middle-aged' , 'old']
customers['age_cat'] = pd.cut(customers['age'], bins=bins, labels=labels, include_lowest=True)
customers_age = customers[['customer_id' , 'age_cat']].merge(transactions[['customer_id' , 'season']] , on = 'customer_id').groupby(['season' , 'age_cat']).agg({'age_cat': 'count'}).rename(columns={'age_cat': 'quantity'}).reset_index()

# Save csv
customers_age.to_csv("data/customers_age.csv",index=False)

In [14]:
# Colors per season
colors_season = transactions[['season' , 'article_id']].merge(articles[['article_id' , 'perceived_colour_master_name']] , on = 'article_id').groupby(['season' , 'perceived_colour_master_name']).agg({'perceived_colour_master_name': 'count'}).rename(columns={'perceived_colour_master_name': 'quantity'}).reset_index()
# Save csv
colors_season.to_csv("data/colors_season.csv",index=False)

Product recommendation

In [21]:
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm

Items bought by customer_id

In [224]:
transactions_agg_customer = transactions.groupby('customer_id')['article_id'].apply(list).reset_index(name='articles')

# Save csv
transactions_agg_customer.to_csv("data/transactions_agg_customer.csv",index=False)

In [225]:
transactions_agg_customer['count_articles']=transactions_agg_customer['articles'].apply(lambda x: len(x))
transactions_agg_customer

,customer_id,articles,count_articles
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,[568601043],1
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,[794321007],1
2,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,"[448509014, 719530003]",2
3,000172a9c322560c849754ffbdfdb2180d408aa7176b94...,"[685814001, 685814001, 685814001]",3
4,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,"[923134003, 835801001, 923134005, 865929003, 5...",10
...,...,...,...
189505,fffd0248a95c2e49fee876ff93598e2e20839e51b9b767...,"[509091057, 859737002, 573085028, 926745002, 8...",6
189506,fffd870c6324ad3bda24e4d6aeae221c199479086bfdfd...,"[750423010, 761269001]",2
189507,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,"[748269009, 803757023, 881919001, 748269009, 9...",11
189508,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,"[713997002, 557599022, 804992033, 791587007, 7...",6


In [226]:
transactions_agg_customer=transactions_agg_customer[transactions_agg_customer['count_articles']> 12].reset_index(drop=True)

In [227]:
transactions_agg_customer

,customer_id,articles,count_articles
0,00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...,"[867966009, 935892001, 907149001, 918171001, 9...",18
1,000fb6e772c5d0023892065e659963da90b1866035558e...,"[831684001, 871519008, 871519008, 871519008, 9...",20
2,00357b192b81fc83261a45be87f5f3d59112db7d117513...,"[904961001, 880815006, 804916006, 880815006, 8...",14
3,00465ec96dd32dca19f85108cbce142de6667a7ace8208...,"[566140001, 862103001, 784247009, 559601019, 8...",14
4,004d932f7a27ac3167c77db81d9cfd89392729e7f7e0d4...,"[678942001, 678942058, 678942047, 783346028, 7...",18
...,...,...,...
8979,ffef8aec5cf011fa1393b40337a5993ce0b7b81af6b322...,"[840909001, 762028005, 789769001, 879139002, 8...",13
8980,fff03ab4ca865dbe1a56bb32a8e41ea23e1b4dfcc4a13f...,"[772902001, 901813001, 907409001, 759814009, 9...",13
8981,fff187d1386edced8ef49b1df0155241943c9c4cc7abbf...,"[867966010, 892558001, 706016003, 914441002, 9...",13
8982,fff2282977442e327b45d8c89afde25617d00124d0f999...,"[919786001, 891322004, 759054001, 891322004, 8...",22


In [229]:
list(transactions_agg_customer.iloc[[5000]]['customer_id'])

['8e0e166ba96a7d4e2fa83ebe7fed15d07c87011085831e4f221b5c2ce14faf93']

Baseline Model

In [61]:
N_MONTHS = 1

window = relativedelta(months=N_MONTHS)
last_date = transactions['t_dat'].max()

threshold = last_date - window

mask = transactions['t_dat'] > threshold
transactions_baseline = transactions[mask]

purchase_dict = {} # Dict that contains each article the user bought and the count of times it was bought

for x in zip(transactions['customer_id'], transactions['article_id']):
    cust_id, art_id = x
    
    if cust_id not in purchase_dict:
        purchase_dict[cust_id] = {}
    purchase_dict[cust_id][art_id] = purchase_dict[cust_id].get(art_id, 0) + 1
    
# List of the most bought articles for all users
best_ever = list(transactions['article_id'].value_counts().index)

# Save
import pickle
f = open("data/best_ever.pkl", "wb")
pickle.dump(best_ever, f)
f.close()

f = open("data/purchase_dict.pkl", "wb")
pickle.dump(purchase_dict, f)
f.close()

Collaborative filtering

In [74]:
# Preprocessing

# Get the transactions from September 1, 2020 on
transactions_copy = transactions.copy()
transactions = transactions_copy[transactions_copy['t_dat'] > '2020-08-31'].sort_values(by=['customer_id'])

# Count the number of same articles bought by the same person and convert to dataframe
counts_df = transactions.groupby(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id']).size()
counts_df = counts_df.to_frame()
counts_df.reset_index(inplace=True)
small_counts = counts_df.rename(columns={0: 'count'})

# Transactions file after September 1, 2020 while the number of same articles bought by each person
small_counts = small_counts.sort_values('customer_id')

from scipy.sparse import csr_matrix, dok_matrix
from pandas.api.types import CategoricalDtype

# Auxiliary function
def to_dense(array):
    """
    Converts a spare matrix (where a lot of elements are zero) to a dense 
    array (an array where the elements are all sequential starting at index 0). 
    
    :param array: Matrix to be converted
    :return: Dense array
    """
    try:
        array = array.todense()
    except:
        pass
    
    return np.array(array).squeeze()

# Create sparse matrix user-item 
def build_counts_table(df):
    """
    Gives an sparse matrix where the columns and the items and the rows the customer. 
    The value is the number of times that a customer has bought an item. 
    
    :param df: original dataframe with transactions
    :return: 
        * Sparse matrix
        * Customer ids corresponding to each row
        * Items ids corresponding to each column
    """
    # Get customer ids and item ids
    customer_ids = CategoricalDtype(sorted(df.customer_id.unique()), ordered=True)
    item_ids = CategoricalDtype(sorted(df.article_id.unique()), ordered=True)

    # Get sparse matrix
    row = df.customer_id.astype(customer_ids).cat.codes
    col = df.article_id.astype(item_ids).cat.codes
    sparse_matrix = csr_matrix((df["count"], (row, col)), \
                           shape=(customer_ids.categories.size, item_ids.categories.size))

    return sparse_matrix, customer_ids, item_ids

# Get sparse matrix for the transactions from Sept 1, 2020
counts, indexes, columns = build_counts_table(small_counts)

# Number of rows is number of customers, number of columns is number of articles
print(counts.shape)

# Get the ids of the top n customers
def top_active_customers(counts, indexes, columns, n):
    """
    Returns the id of the top n customers, in terms of items bought 
    
    :param counts, indexes, columns: Tuple returned by build_counts_table
    :param n: Number of users
    :return: Series of customerID of the top users
    """
    # Operate with the sparse matrix, convert to dense the result
    sums = to_dense(counts.sum(axis=1))
    # Get indices
    indices = sums.argsort()
    return indexes.categories[indices[-n:]]

# Get the ids of the top n articles
def top_bought_articles(counts, indexes, columns, n):
    """
    Returns the top n most bought items
    
    :param counts, indexes, columns: Tuple returned by build_counts_table
    :param n: Number of items
    :return: Series of itemID of the top items
    """
    # Operate with the sparse matrix, convert to dense the result
    sums = to_dense(counts.sum(axis=0))
    # Get indices
    indices = sums.argsort()
    return columns.categories[indices[-n:]]

# Get the top 5,000 articles and users from the transactions after Sept 1, 2020
top_customers = top_active_customers(counts, indexes, columns, 5000)
top_items = top_bought_articles(counts, indexes, columns, 5000)

# Transactions from Sept 1, 2020
s = small_counts.copy()
# Transactions from Sept 1, 2020 that include one of the most bought 5,000 items
s = s[s.article_id.isin(top_items)]

# Transactions from Sept 1, 2020 that include one of the most bought 5,000 items and belong to one of the most active 5,000 customers
s = s[s.customer_id.isin(top_customers)]

# Drop the non-relevant info
s = s.drop(s.columns[[0, 3, 4]], axis=1)

(189510, 26252)


In [79]:
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

# Creation of the matrix
X_transactions_matrix = pd.pivot_table(s,values='count',index='customer_id',columns='article_id')
# We fill Na values with 0
X_transactions_matrix = X_transactions_matrix.fillna(0)

# Merge with interesting columns from items dataset
X_transactions = s.merge(articles[['article_id','colour_group_code','index_group_name','product_group_name']],on=['article_id'])
X_transactions.head()

product_cat = X_transactions[['article_id','colour_group_code','index_group_name','product_group_name']].drop_duplicates('article_id')
product_cat = product_cat.sort_values(by='article_id')

# First, for the numerical feature 'colour_group_code' we create an euclidian matrix based on euclidean distances between items
color_cat_matrix = np.reciprocal(euclidean_distances(np.array(product_cat['colour_group_code']).reshape(-1,1))+1)
euclidean_matrix = pd.DataFrame(color_cat_matrix,columns=product_cat['article_id'],index=product_cat['article_id'])

# Then, for the categorical feature 'product_group_name' we construct a cosinus similarity matrix
tfidf_vectorizer = TfidfVectorizer()
doc_term = tfidf_vectorizer.fit_transform(list(product_cat['product_group_name']))
dt_matrix = pd.DataFrame(doc_term.toarray().round(3), index=[i for i in product_cat['article_id']], columns=tfidf_vectorizer.get_feature_names_out())
cos_similar_matrix = pd.DataFrame(cosine_similarity(dt_matrix.values),columns=product_cat['article_id'],index=product_cat['article_id'])

# Then, for the categorical feature 'index_group_name' we also construct a cosinus similarity matrix
tfidf_vectorizer = TfidfVectorizer()
doc_term = tfidf_vectorizer.fit_transform(list(product_cat['index_group_name']))
dt_matrix1 = pd.DataFrame(doc_term.toarray().round(3), index=[i for i in product_cat['article_id']], columns=tfidf_vectorizer.get_feature_names_out())
dt_matrix1 = dt_matrix1 + 0.01
cos_similar_matrix1 = pd.DataFrame(cosine_similarity(dt_matrix1.values),columns=product_cat['article_id'],index=product_cat['article_id'])

# Finally, by multiplying the following three matrices we obtain our final Item-Item Similarity Matrix
similar_matrix = cos_similar_matrix.multiply(euclidean_matrix).multiply(cos_similar_matrix1)

content_matrix = X_transactions_matrix.dot(similar_matrix)
# We then normalize the obtained matrix
std = MinMaxScaler(feature_range=(0, 1))
std.fit(content_matrix.values)
content_matrix = std.transform(content_matrix.values)

# We then create a table with the predicted_count for each couple (customer_id, article_id)
content_matrix = pd.DataFrame(content_matrix,columns=sorted(X_transactions['article_id'].unique()),index=sorted(X_transactions['customer_id'].unique()))
# The trained User-Item Matrix is then re-converted back into a data frame
content_df = content_matrix.stack().reset_index()
content_df = content_df.rename(columns={'level_0':'customer_id','level_1':'article_id',0:'predicted_count'})

# Save csv
content_df.to_csv("data/content_df.csv",index=False)

Rule-based algorithm

In [137]:
Recent_transactions=transactions[['t_dat','customer_id','article_id']]
last_ts = Recent_transactions['t_dat'].max()
# we then convert the customer_id to int
Recent_transactions['customer_id']=Recent_transactions['customer_id'].apply(lambda x: int(x[-16:], 16))

<ipython-input-137-f53c0c021f62>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Recent_transactions['customer_id']=Recent_transactions['customer_id'].apply(lambda x: int(x[-16:], 16))


In [138]:
# We create a temporary dataframe with the last day of the week for each date
tmp = Recent_transactions[['t_dat']].copy()
# We get the day of week for each date : Monday = 0, Tuesday = 1 and so on
tmp['dow'] = tmp['t_dat'].dt.dayofweek
# We truncated t_dat into the Tuesday of t_dat week and we save it as ldbw
tmp['ldbw'] = tmp['t_dat'] - pd.TimedeltaIndex(tmp['dow'] - 1, unit='D')
# For t_dat from Wednesday until Sunday we add 7 days to get next Tuesday, do nothing for t_dat Monday and Tuesdaytmp.loc[tmp['dow'] >=2 , 'ldbw'] = tmp.loc[tmp['dow'] >=2 , 'ldbw'] + pd.TimedeltaIndex(np.ones(len(tmp.loc[tmp['dow'] >=2])) * 7, unit='D')
Recent_transactions['ldbw'] = tmp['ldbw'].values

<ipython-input-138-654d1fcf31e2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Recent_transactions['ldbw'] = tmp['ldbw'].values


In [139]:
weekly_sales = Recent_transactions.drop('customer_id', axis=1).groupby(['ldbw', 'article_id']).count().reset_index()
weekly_sales = weekly_sales.rename(columns={'t_dat': 'count'})

In [140]:
# We then merge it with our transaction dataset
Recent_transactions = Recent_transactions.merge(weekly_sales, on=['ldbw', 'article_id'], how = 'left')

In [141]:
weekly_sales = weekly_sales.reset_index().set_index('article_id')

# We create the "count_targ" column corresponding to the number of transactions during the last week
Recent_transactions = Recent_transactions.merge(
    weekly_sales.loc[weekly_sales['ldbw']==last_ts, ['count']],
    on='article_id', suffixes=("", "_targ"))

# We then fill the missing values with 0
Recent_transactions['count_targ'].fillna(0, inplace=True)

In [142]:
Recent_transactions['quotient'] = Recent_transactions['count_targ'] / Recent_transactions['count']

In [143]:
# Number of articles to be recommended by customer_id
N=12

# We group the quotient column by article_id and we then rank them by value
target_sales = Recent_transactions.drop('customer_id', axis=1).groupby('article_id')['quotient'].sum()
general_pred = target_sales.nlargest(N).index.tolist()
general_pred = ['0' + str(article_id) for article_id in general_pred]
general_pred_str =  ' '.join(general_pred)

In [144]:
tmp = Recent_transactions.copy()

# x: number of days elapsed from the date Customer A purchased Product B to 2020-09-22
tmp['x'] = ((last_ts - tmp['t_dat']) / np.timedelta64(1, 'D')).astype(int)
tmp['dummy_1'] = 1 
tmp['x'] = tmp[["x", "dummy_1"]].max(axis=1)

In [145]:
# Now that we know the function to modelize the this phenomenon we create the column y 
# y: Value to be decayed by the x 
# with the value of a, b, c and d explained before
a, b, c, d = 2.5e4, 1.5e5, 2e-1, 1e3
tmp['y'] = a / np.sqrt(tmp['x']) + b * np.exp(-c*tmp['x']) - d
tmp['dummy_0'] = 0 
tmp['y'] = tmp[["y", "dummy_0"]].max(axis=1)

In [146]:
# value: y * quotient 
tmp['value'] = tmp['quotient'] * tmp['y'] 
tmp = tmp.groupby(['customer_id', 'article_id']).agg({'value': 'sum'})
tmp = tmp.reset_index()

tmp = tmp.loc[tmp['value'] > 100]
tmp['rank'] = tmp.groupby("customer_id")["value"].rank("dense", ascending=False)
tmp = tmp.loc[tmp['rank'] <= 12]

# We then group tmp by customer_id and rank the articles by their value
purchase_df = tmp.sort_values(['customer_id', 'value'], ascending = False).reset_index(drop = True)
purchase_df['prediction'] = '0' + purchase_df['article_id'].astype(str) + ' '
purchase_df = purchase_df.groupby('customer_id').agg({'prediction': sum}).reset_index()
purchase_df['prediction'] = purchase_df['prediction'].str.strip()

# Save csv
purchase_df.to_csv("data/purchase_df.csv",index=False)
text_file = open("data/general_pred_str.txt", "wt")
n = text_file.write(general_pred_str)
text_file.close()

In [170]:
purchase_df

,customer_id,prediction
0,28847241659200,0925246001
1,116809474287335,0906305002
2,200292573348128,0903861001
3,272412481300040,0922381001 0921906005 0923460002
4,519262836338427,0804992016 0852584006
...,...,...
183112,18446420423308293068,0577512001 0579541001 0736923011 0786304009 08...
183113,18446590778427270109,0751471022 0783346018
183114,18446630855572834764,0898713001 0886966002 0568601045
183115,18446705133201055310,0875784002
